<div style = "text-align:center;">
    <img style = "width:70%;"  src = "https://lh3.googleusercontent.com/proxy/n8SXNVIZEaY35qrZsELDJUKUgo4im-2NAkj7nVV0Wb_2qAZ1UIivF9Yc_mBiXbK0TDgUGkROFQdw7OMOocBhFBCZTIbKNpIN4olH_P1ERjgkn9A0vO5Ng58"/>
    </div>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv")
movies.head()

In [ ]:
credits = pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv")
credits.head()

#### Shape of dataset

In [ ]:
movies.shape, credits.shape

#### Merging dataframe

In [ ]:
credits_renamed = credits.rename(index = str, columns = {"movie_id":"id"})
merged_dataframe = pd.merge(movies,credits_renamed, on = "id")
merged_dataframe.head()

#### Shape of merged dataset

In [ ]:
merged_dataframe.shape

#### Droping the columns  "homepage","title_x","title_y","status","productions_countries"

In [ ]:
merged_dataframe_cleaned = merged_dataframe.drop(["homepage","title_x","title_y","status","production_countries"], axis=1)

In [ ]:
merged_dataframe_cleaned.info()

In [ ]:
merged_dataframe_cleaned.head()

In [ ]:
merged_dataframe_cleaned.head(1)

#### Overview column

In [ ]:
merged_dataframe_cleaned["overview"]

#### import Tfidf vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfy = TfidfVectorizer(min_df=3, max_features = None, strip_accents="unicode",
                      analyzer="word", token_pattern=r"\w{1,}", ngram_range=(1,3), stop_words = "english")
tfv_matrix = tfy.fit_transform(merged_dataframe_cleaned["overview"].values.astype(str))
print(tfv_matrix)
print(tfv_matrix.shape)

#### Using sigmoid kernel function on tfidf vectorization 

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig_ker = sigmoid_kernel(tfv_matrix, tfv_matrix)
sig_ker

#### Building a series reversing indices and movie titles

In [ ]:
indices = pd.Series(merged_dataframe_cleaned.index, merged_dataframe_cleaned["original_title"]).drop_duplicates(keep="first",inplace = False)
indices

In [ ]:
indices.shape

In [ ]:
indices["John Carter"], sig_ker[indices["John Carter"]]

In [ ]:
 len(sig_ker[indices["John Carter"]])

#### Defining function for recomendation

In [ ]:
def recommend_movie(title, sig=sig_ker):
    index = indices[title]
    sig_scores = list(enumerate(sig[index]))
    sig_scores = sorted(sig_scores, key = lambda x: x[1],reverse=True)
    movie_indices = [i[0] for i in sig_scores]
    sig_scores = sig_scores[1:11]
    return merged_dataframe_cleaned["original_title"].iloc[movie_indices]

In [ ]:
recommend_movie("Avatar")

<p style="font-weight:bold;font-size:20px;"> Thank you for looking my notebook, see you later 👋  </p>